Pseudopotential Test.

In [ ]:
from wizard.io import read_xyz
from ase.calculators.vasp import Vasp
import shutil
import os

frames = read_xyz('test.xyz')
for i in range(len(frames)):
    atoms = frames[i]
    dir = 'Your_Path'
    if os.path.exists(dir):
        shutil.rmtree(dir)
    calc = Vasp(directory=dir,
                encut=500,                  # ENCUT
                ediff=1e-6,                 # EDIFF
                kspacing=0.15,              # KSPACING
                sigma=0.1,                  # SIGMA
                ismear=1,                   # ISMEAR
                gga='PE',                   # GGA
                prec='Accurate',            # PREC
                lasph=True,                 # LASPH
                nelmin=4,                   # NELMIN
                nelm=200,                   # NELM
                kpar=4,                     # KPAR
                ncore=20,                   # NCORE
                lwave=False)                # LWAVE      
    calc.write_input(atoms)                 # Write input files

Ground State Energy Test.

In [ ]:
from wizard.atoms import SymbolInfo
from ase.calculators.vasp import Vasp
import shutil
import os

symbol_infos = [
    SymbolInfo('Mo', 'bcc', 3.163),
    SymbolInfo('Nb', 'bcc', 3.307),
    SymbolInfo('Ta', 'bcc', 3.321),
    SymbolInfo('V', 'bcc', 2.997),
    SymbolInfo('W', 'bcc', 3.185)
]

for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms()
    dir=f'Your_Path/{symbol_info.formula}_coh'
    if os.path.exists(dir):
        shutil.rmtree(dir)
    calc = Vasp(directory= dir,  
            encut=500,                  # ENCUT
            ediff=1e-6,                 # EDIFF
            kspacing=0.15,              # KSPACING
            sigma=0.1,                  # SIGMA
            ismear=1,                   # ISMEAR
            ibrion=2,                   # IBRION
            isif=3,                     # ISIF
            nsw=200,                    # NSW
            gga='PE',                   # GGA
            prec='Accurate',            # PREC
            lasph=True,                 # LASPH
            nelmin=4,                   # NELMIN
            nelm=200,                   # NELM
            kpar=4,                     # KPAR
            lwave=False)                # LWAVE
    calc.write_input(atoms)             # Write input files

Mono_Vacancy Calculations.

In [ ]:
from wizard.atoms import SymbolInfo
from wizard.calculator import MaterialCalculator
from wizard.frames import MultiMol
from wizard.io import read_xyz
from pynep.calculate import NEP

symbol_infos = [
    SymbolInfo('Mo', 'bcc', 3.163),
    SymbolInfo('Nb', 'bcc', 3.308),
    SymbolInfo('Ta', 'bcc', 3.321),
    SymbolInfo('V', 'bcc', 2.997),
    SymbolInfo('W', 'bcc', 3.185)
]
for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms()
    calc = NEP('nep.txt')
    material_calculator = MaterialCalculator(atoms, calc, symbol_info.formula, symbol_info.structure)
    material_calculator.formation_energy_vacancy(supercell=(3,3,3))

frames = read_xyz('MaterialProperties.xyz')
for i in range(len(frames)):
    atoms = frames[i]
    dir=f'Your_Path/vacancies/{i}'
    calc = Vasp(directory= dir,  
            encut=500,                  # ENCUT
            ediff=1e-6,                 # EDIFF
            kspacing=0.15,              # KSPACING
            sigma=0.1,                  # SIGMA
            ismear=1,                   # ISMEAR
            ibrion=2,                   # IBRION
            isif=3,                     # ISIF
            nsw=200,                    # NSW
            gga='PE',                   # GGA
            prec='Accurate',            # PREC
            lasph=True,                 # LASPH
            nelmin=4,                   # NELMIN
            nelm=200,                   # NELM
            kpar=4,                     # KPAR
            lwave=False)                # LWAVE
    atoms.calc = calc
    calc.write_input(atoms)             # Write input files

Self-Interstitial Calculations.

In [ ]:
from wizard.atoms import SymbolInfo
from wizard.calculator import MaterialCalculator
from wizard.frames import MultiMol
from wizard.io import read_xyz
from pynep.calculate import NEP

symbol_infos = [
    SymbolInfo('Mo', 'bcc', 3.163),
    SymbolInfo('Nb', 'bcc', 3.308),
    SymbolInfo('Ta', 'bcc', 3.321),
    SymbolInfo('V', 'bcc', 2.997),
    SymbolInfo('W', 'bcc', 3.185)
]
sia_vectors = [(1,1,1),(1,0,0),(1,1,0)]
for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms()
    calc = NEP('nep.txt')
    material_calculator = MaterialCalculator(atoms, calc, symbol_info.formula, symbol_info.structure)
    for vector in sia_vectors:
        material_calculator.formation_energy_sia(vector, supercell=(4,4,4))

frames = read_xyz('MaterialProperties.xyz')
for i in range(len(frames)):
    atoms = frames[i]
    dir=f'Your_Path/sias/{i}'
    calc = Vasp(directory= dir,  
            encut=500,                  # ENCUT
            ediff=1e-6,                 # EDIFF
            kspacing=0.15,              # KSPACING
            sigma=0.1,                  # SIGMA
            ismear=1,                   # ISMEAR
            ibrion=2,                   # IBRION
            isif=3,                     # ISIF
            nsw=200,                    # NSW
            gga='PE',                   # GGA
            prec='Accurate',            # PREC
            lasph=True,                 # LASPH
            nelmin=4,                   # NELMIN
            nelm=200,                   # NELM
            kpar=4,                     # KPAR
            lwave=False)                # LWAVE
    atoms.calc = calc
    calc.write_input(atoms)             # Write input files

Loading VASP Results.

In [ ]:
"""
# -----------------------------------------------------------------------------
# The following code is sourced from
# GPUMD:
# https://github.com/brucefan1983/GPUMD
#
# Copyright (c) 2022 wang laosi, licensed under MIT License
# -----------------------------------------------------------------------------
"""

import os, sys
import numpy as np
from ase.io import read, write
from ase import Atoms, Atom
from tqdm import tqdm


def Convert_atoms(atom):
    # 1 eV/Å^3 = 160.21766 GPa
    xx,yy,zz,yz,xz,xy = -atom.calc.results['stress']*atom.get_volume() # *160.21766 
    atom.info['virial'] = np.array([(xx, xy, xz), (xy, yy, yz), (xz, yz, zz)])
    atom.calc.results['energy'] = atom.calc.results['free_energy']
    del atom.calc.results['stress']
    del atom.calc.results['free_energy']


def find_vasprun(start_path='.'):
    result = []
    for root, dirs, files in os.walk(start_path):
        if 'vasprun.xml' in files:
            result.append(os.path.join(root, 'vasprun.xml'))
    return result

Path = 'dir_name'
file_list = find_vasprun('Your_Path/'+ Path)

cnum = 0     # total number of configuration
atoms_list, err_list = [], []
for dir_name in tqdm(file_list):
    try:
        atoms = read(dir_name.strip('\n'), index=":")
    except:
        err_list.append(dir_name)
        continue
    for ai in range(len(atoms)):
        Convert_atoms(atoms[ai])
        atoms_list.append(atoms[ai])
    cnum += len(atoms)

write('dataset/' + Path + '.xyz', atoms_list, format='extxyz')
print('The total number of configurations is: {} \n'.format(cnum))

if err_list:
    print("The list of failed calculation files is as follows.")
    for err_dirname in err_list:
        print(err_dirname)
else:
    print("All calculations are successful!")

Merge Training Sets.

In [ ]:
from wizard.io import read_xyz
from wizard.frames import MultiMol
import glob

files = glob.glob('dataset/*.xyz')
train_set = []
for file in files:
    frames = read_xyz(file)
    for atoms in frames:
        atoms.info['config_type'] = file.split('/')[-1][:-4]
    train_set.extend(frames)
MultiMol(train_set).dump('dataset/train.xyz')